In [1]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
text = text[300000:800000]

In [4]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [5]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

In [6]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [8]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

In [9]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.01))

model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - loss: 2.4819
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 1.7889
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - loss: 1.6113
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 1.5306


In [12]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [14]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))

ed withal.

nurse:
anon, anon!
come, let the senter the sention to the heads.

henry bolingbroke:
my lord of his sunt the senter with her hate.

henry bolingbroke:
i am not the senter hath the sention.

paris:
i see the senter of his son,
and the senting the sention the sentle the sence
and the senter the senter and the senter.

king henr
 sir, of such a man, who hath a
daughter thou shall not my father's carrow
should in the compose, i am the sends is the secking.

henry bolingbroke:
i am death is confult himself,
and the best not it shall the best hate,
and the best tell the wind the mounting the tone.

romeo:
and below what beceits for the mind and thee shall shall no b
k:
sweet rest his soul! fly, lords, and the founts.

henry bolingbroke:
you did now thee to the took and her
his heart is the heart, in a sears the arrer!

romeo:
come on my son the fill, and she conceing thee love?

polixenes:
i am too fount and the man and will it,
and grave their love be with my bands some
past th